In [48]:
from google.cloud import firestore
import os
import pandas as pd

In [49]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='../acciones-usd-real-firebase-adminsdk-rfxvv-c888549652.json'

# Inicializo firestore
db = firestore.Client()

In [50]:
list_tickers = []
try:
    # Consulto los tickers y los guardo en una lista
    tickers = db.collection('acciones').stream()
    for ticker in tickers:
        list_tickers.append(ticker.to_dict())
    
    # Convierto la lista en un data frame
    df_tickers = pd.DataFrame(list_tickers)
except Exception as e:
    print(f'No fue posible consultar los datos, error: {e}')


In [51]:
from datetime import datetime
import requests

# Funcion para obtener el valor del dolcar CCL al dia de la fecha
def valor_ccl():
    FECHA_INICIO = datetime.now()
    FECHA_FINAL = datetime.now()
    # URL de la API
    url_ccl = f"https://mercados.ambito.com//dolarrava/cl/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}"
    #url_mep = f'https://mercados.ambito.com//dolarrava/mep/grafico/{FECHA_INICIO.date()}/{FECHA_FINAL.date()}'

    # Realizar una solicitud GET
    response = requests.get(url_ccl)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        # Parsear el contenido JSON de la respuesta
        data = response.json()
        return data[1][1]
    else:
        print(f"Error: {response.status_code}")

#dolar_ccl = valor_ccl()
dolar_ccl = 1480
print(dolar_ccl)

1480


In [52]:
import yfinance as yf

# Las convierto en lista para poder descargar sus datos y los descargo con yfinance
list_tickers = df_tickers['Ticker'].to_list()
yf_t = yf.download(list_tickers, period='1d', interval='2m')['Adj Close']

# Elijo el ultimo registro
yf_t = yf_t.iloc[-1]

# Creo una columna donde agrego esos precios mapeandolos con el indice de la serie anterior ya que son el mismo
df_tickers['Cierre_ARS'] = df_tickers['Ticker'].map(yf_t).round(2)

# Agrego otra columna donde se ve la ganancia/perdida en ARS y una donde tengo el total invertido
df_tickers['Total invertido ARS'] = (df_tickers['Cierre_ARS'] * df_tickers['Cantidad']).round(2)
df_tickers['Gan/Perd ARS'] = ((df_tickers['Cierre_ARS'] * df_tickers['Cantidad']) - (df_tickers['PPC'] * df_tickers['Cantidad'])).round(2)

# Hago lo mismo pero en USD
df_tickers['Total invertido USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2)
df_tickers['Gan/Perd USD'] = (df_tickers['Cantidad'] * (df_tickers['Cierre_ARS'] / dolar_ccl)).round(2) - ((df_tickers['PPC'] / dolar_ccl) * df_tickers['Cantidad']).round(2)
hoy = datetime.now()
df_tickers

[*********************100%%**********************]  2 of 2 completed


,Ticker,PPC,Cantidad,Cierre_ARS,Total invertido ARS,Gan/Perd ARS,Total invertido USD,Gan/Perd USD
0,AGRO.BA,59.70,280,62.9,17612.0,896.00,11.90,0.61
1,NVDA.BA,7163.88,21,7690.0,161490.0,11048.52,109.11,7.46


In [53]:
# Guardo un excel con los datos
df_tickers.to_excel(f'informe-acciones-usd.xlsx')

In [54]:
import plotly.express as px
import plotly.graph_objs as go

# Calculo la ganancia total en USD
total_usd = (df_tickers['Gan/Perd USD'].sum()).round(2)

# Grafico la participación x accion en usd
fig_pie = px.pie(df_tickers, values='Total invertido USD', names='Ticker', title='Participación en USD por acción')
fig_pie.update_traces(textposition='outside', textinfo='percent+label')
fig_pie.add_annotation(text=f'La ganancia neta es de {total_usd} USD', x=-0.3, y=1.1, showarrow=False, font=dict(size=10, color='gray'), xref='paper', yref='paper')
fig_pie.show()

# Ganancia/perdida en dolares de cada accion
fig_bars = go.Figure(data=[
    go.Bar(name='Total invertido USD', x=df_tickers['Ticker'], y=df_tickers['Total invertido USD']),
    go.Bar(name='Gan/Perd USD', x=df_tickers['Ticker'], y=df_tickers['Gan/Perd USD'])
])
# Change the bar mode
fig_bars.update_layout(barmode='group', title='Total invertido y Ganancias en USD por accion')
fig_bars.show()